In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Imports
# -----------------------------------------------------------------------------------------------------------------------
from ipywidgets import (Layout, Button, Label, Password, Text, VBox, HBox, Output, HTML, BoundedFloatText, 
                        BoundedIntText, Dropdown, Checkbox, IntText)
from IPython.display import display
import matplotlib.pyplot as plt
import json
from dashchecks import *

In [2]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Global Variables
# -----------------------------------------------------------------------------------------------------------------------
class active_user:
    def __init__(self, eid = -1, hid = -1, position = -1, fname = -1, lname = -1, lid = -1):
        self.eid = eid
        self.hid = hid
        self.position = position
        self.firstname = fname
        self.lastname = lname
        self.lid = lid
user = active_user()

In [4]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Create Chain
# -----------------------------------------------------------------------------------------------------------------------
def CreateChain():
    def post_chain(button):
        with output:
            chain = Post_Chain(cname.value, fallmkup.value, springmkup.value, summermkup.value, wintermkup.value)
            if chain.status_code == 201:
                output.clear_output()
                print("Chain created successfully")
                print("Chain Data:")
                print(chain.json())  # Print the chain data
            else:
                output.clear_output()
                print(f"Invalid Chain Data")

    operation = HTML(f"<p style='font-size:24px;'>Create Chain</p>")
    
    cname = Text(placeholder="---", description='Chain Name')
    fallmkup = BoundedFloatText(min=0, step=0.01, description='Fall Mkup')
    springmkup = BoundedFloatText(min=0, step=0.01, description='Spring Mkup')
    summermkup = BoundedFloatText(min=0, step=0.01, description='Summer Mkup')
    wintermkup = BoundedFloatText(min=0, step=0.01, description='Winter Mkup')
    create_button = Button(description='Create Chain')
    create_button.on_click(post_chain)
    
    create_box = VBox([operation, cname, fallmkup, springmkup, summermkup, wintermkup, create_button])
    display(create_box)
    
    output = Output()
    display(output)



In [5]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Delete Chain
# -----------------------------------------------------------------------------------------------------------------------
def DeleteChain():
    def delete_chain(b):
        with output:
            chain = Delete_Chain(chid.value)
            if chain.status_code == 200:
                output.clear_output()
                print("Chain deleted successfully")
            elif chain.status_code == 404:
                output.clear_output()
                print("Chain not found")
            elif chain.status_code == 400:
                output.clear_output()
                print("Chain is referenced")
            else:
                output.clear_output()
                print("Delete Failed")
            
    operation = HTML(f"<p style='font-size:24px;'>Delete Chain</p>")
    
    chid = BoundedIntText(min=-1,step=1, description="Chain Id")
    delete_button = Button(description="Delete Chain")
    delete_button.on_click(delete_chain)
    
    create_box = VBox([operation, chid, delete_button])
    display(create_box)
    
    output = Output()
    display(output)



In [3]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Dashboard
# -----------------------------------------------------------------------------------------------------------------------
# Different styles definitions:
login_button_style = {"description_border_color": "lightblue", "font_weight": "bold", "button_color": "lightblue"}
input_style = {"description_width": "initial"}
header_style = {"font_weight": "bold", "color": "black", "background-color": "lightblue", "padding": "10px", "text-align": "center"}
welcome_style = {"font_weight": "bold", "color": "#008080", "background-color": "lightblue", "padding": "10px", "text-align": "center", "font-size": "28px"}

# Log In Fields
username = Text(placeholder="Username", style=input_style)
password = Password(placeholder="Password", style=input_style)

# Create the buttons
login_button = Button(description="Log In", style=login_button_style, icon="check", button_style="")
sign_up_button = Button(description="Sign Up", style=login_button_style, icon="plus", button_style="")
back_button = Button(description="Back", style=login_button_style, icon="arrow-left", button_style="")
create_button = Button(description="Create", style=login_button_style, icon="check", button_style="")
logout_button = Button(description="Log Out", style=login_button_style, icon="times", button_style="")

# Create output widget
output = Output()

# Create the VirtualBox for the login
login_form = VBox([
    Label(value="Hello, welcome!", layout=Layout(margin="10px"), style=welcome_style),
    username,
    password,
    VBox([login_button], layout=Layout(justify_content="center")),
    VBox([sign_up_button], layout=Layout(justify_content="center")),
    output,
], layout=Layout(justify_content="center", align_items="center"))

# SignUp Fields
Employee_ID = IntText(description="Employee ID", style=input_style)
First_Name = Text(placeholder="First Name", style=input_style)
Last_Name = Text(placeholder="Last Name", style=input_style)
signup_username = Text(placeholder="Username", style=input_style)
signup_password = Password(placeholder="Password", style=input_style)

# Create the VirtualBox for the SignUp
signup_form = VBox([
    Label(value="Create an Account", layout=Layout(margin="10px"), style=welcome_style),
    Employee_ID,
    First_Name,
    Last_Name,
    signup_username,
    signup_password,
    VBox([create_button], layout=Layout(justify_content="center")),
    VBox([back_button], layout=Layout(justify_content="center")),
    output
], layout=Layout(justify_content="center", align_items="center"))

# Create the VirtualBox for the logout
logout_form = VBox([
    Label(value=f"Welcome!", layout=Layout(margin="10px"), style=welcome_style),
    VBox([logout_button], layout=Layout(justify_content="center")),
    output,
], layout=Layout(justify_content="center", align_items="center"))

# Function to handle login button click
def login_button_clicked(button):
    username_input=username.value
    password_input=password.value
    response = Check_Login(username_input, password_input)
    if response.status_code==200:
         with output:
            output.clear_output()
            eid_substracted= response.json()['login']['eid'] 
            userinfo_response = Check_Employeee(eid_substracted).json()['employee']
            user_position = userinfo_response['position']    
            hotel_id = userinfo_response['hid']
            fname = userinfo_response['fname']
            lname = userinfo_response['lname']
            # print(eid_substracted)
            # print(user_position)
            # print(hotel_id)
            user.eid = eid_substracted
            user.position = user_position
            user.hotel_id = hotel_id
            user.fname = fname
            user.lname = lname
            # print(user.eid)
            # print(user.position)
            # print(user.hotel_id)
            print(f"Login successful! {user.eid}, {user.position}")
            print(f"Welcome {user.fname} {user.lname}")
            form_container.children = [logout_form]
            Display_controller()
    else:
        with output:
            output.clear_output()
            print("Login failed!")
    
# Function to handle signup button click
def signup_button_clicked(button):
    form_container.children = [signup_form]
    with output: 
        output.clear_output()
    
# Function to handle back button click
def back_button_clicked(button):
    # Reset login fields
    username.value = ""
    password.value = ""
    # Reset signup fields
    Employee_ID.value = 0
    First_Name.value = ""
    Last_Name.value = ""
    signup_username.value = ""
    signup_password.value = ""
    form_container.children = [login_form]
    with output:
        output.clear_output()
    
# Function to handle create button click
def create_button_clicked(button):
    with output:
        # Checks whether an employee with these credentials exists.
        employee = Check_Employee(Employee_ID.value, First_Name.value, Last_Name.value)
        if employee.status_code == 200:
            # Create a new login for the employee. 
            post_login = Post_Login(Employee_ID.value, signup_username.value, signup_password.value)
            if post_login.status_code == 201:
                output.clear_output()
                print("Account created successfully!")
                print(post_login.json()['Login'])
            else:
                output.clear_output()
                print("Account creation failed!")
        elif employee.json()['Error'] == "Not Found":
            output.clear_output()
            print("Employee not found!")
        elif employee.json()['Error'] == "First name or last name not valid":
            output.clear_output()
            print("First name or last name not valid with employee ID")
        else:
            output.clear_output()
            print(f"Error")
            
# Function to handle logout button click
def logout_button_clicked(button):
    # Reset login fields
    username.value = ""
    password.value = ""
    # Reset signup fields
    Employee_ID.value = 0
    First_Name.value = ""
    Last_Name.value = ""
    signup_username.value = ""
    signup_password.value = ""
    form_container.children = [login_form]
    with output:
        output.clear_output()

# Button clicks and their events.
login_button.on_click(login_button_clicked)
sign_up_button.on_click(signup_button_clicked)
back_button.on_click(back_button_clicked)
create_button.on_click(create_button_clicked)
logout_button.on_click(logout_button_clicked)

# Create layout (Virtual Boxes)
def create_layout():
    login_form_box = VBox([
        Label(value="Hello, welcome!", layout=Layout(margin="10px"), style=welcome_style),
        username,
        password,
        VBox([login_button], layout=Layout(justify_content="center")),
        VBox([sign_up_button], layout=Layout(justify_content="center")),
        output,
    ], layout=Layout(justify_content="center", align_items="center"))
    
    signup_form_box = VBox([
        Label(value="Create an Account", layout=Layout(margin="10px"), style=welcome_style),
        Employee_ID,
        First_Name,
        Last_Name,
        signup_username,
        signup_password,
        VBox([create_button], layout=Layout(justify_content="center")),
        VBox([back_button], layout=Layout(justify_content="center")),
        output
    ], layout=Layout(justify_content="center", align_items="center"))
    
    logout_form_box = VBox([
        Label(value=f"Welcome!", layout=Layout(margin="10px"), style=welcome_style),
        VBox([login_button], layout=Layout(justify_content="center")),
        output,
    ], layout=Layout(justify_content="center", align_items="center"))
    
    form_container = HBox([login_form_box], layout=Layout(justify_content="center", align_items="center"))
    
    header = HTML("<div style='background-color:lightblue; padding:10px; text-align:center; overflow:hidden;'><p style='font-weight:bold; color:black; font-size:24px;'>HotelAnalytics Systems</p><p style='font-weight:bold; color:black; font-size:18px;'>DB Beasts</p></div>") 
    footer = HTML("<div style='background-color:lightblue; padding:10px; text-align:center; overflow:hidden;'><a style='text-decoration:none; color:black;'>Db Beasts Members:</a> <i class='fas fa-user' style='background-color: #00868B; color: white; border-radius: 50%; padding: 2px 6px; font-weight: bold;'></i><a href='https://github.com/YanehRuiz' target='_blank' style='text-decoration:none; color:black;'> Yanishka</a> <i class='fas fa-user' style='background-color: #00868B; color: white; border-radius: 50%; padding: 2px 6px; font-weight: bold;'></i><a href='https://github.com/gaac-rum' target='_blank' style='text-decoration:none; color:black;'> Gabriel</a> <i class='fas fa-user' style='background-color: #00868B; color: white; border-radius: 50%; padding: 2px 6px; font-weight: bold;'></i><a href='https://github.com/Axel-Negron' target='_blank' style='text-decoration:none; color:black;'> Axel  </a><i class='fas fa-user' style='background-color: #00868B; color: white; border-radius: 50%; padding: 2px 6px; font-weight: bold;'></i><a href='https://github.com/samiralirivera' target='_blank' style='text-decoration:none; color:black;'> Samir</a></div>")    
    background = HTML("<img src='https://i.imgur.com/sJ8W01b.jpeg' style='width:100%; height:90vh; object-fit:cover;'>")   
    return header, form_container, background, footer

# Display header, layout, and backgrounde
header, form_container, background, footer = create_layout()
display(header)
display(form_container)
display(background)
display(footer)

HTML(value="<div style='background-color:lightblue; padding:10px; text-align:center; overflow:hidden;'><p styl…

HTML(value="<img src='https://i.imgur.com/sJ8W01b.jpeg' style='width:100%; height:90vh; object-fit:cover;'>")

HTML(value="<div style='background-color:lightblue; padding:10px; text-align:center; overflow:hidden;'><a styl…

In [6]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Update Chain
# -----------------------------------------------------------------------------------------------------------------------
def UpdateChain():
    def put_chain(b):
        with output:
            chain = Put_Chain(chid.value, cname.value, fallmkup.value, springmkup.value, summermkup.value, wintermkup.value)
            if chain.status_code == 202:
                output.clear_output()
                print("Chain updated successfully")
                print("Chain Data:")
                print(chain.json())
            elif chain.status_code == 404:
                output.clear_output()
                print("Chain not found")
            else:
                output.clear_output()
                print("Invalid Chain Data")
            
    operation = HTML(f"<p style='font-size:24px;'>Update Chain</p>")
    
    chid = BoundedIntText(min=-1,step=1, description="Chain Id")
    cname = Text(placeholder="---", description='Chain Name')
    fallmkup = BoundedFloatText(min=0, step=0.01, description='Fall Mkup')
    springmkup = BoundedFloatText(min=0, step=0.01, description='Spring Mkup')
    summermkup = BoundedFloatText(min=0, step=0.01, description='Summer Mkup')
    wintermkup = BoundedFloatText(min=0, step=0.01, description='Winter Mkup')
    update_button = Button(description="Update Chain")
    update_button.on_click(put_chain)
    
    create_box = VBox([operation, chid, cname, fallmkup, springmkup, summermkup, wintermkup, update_button])
    display(create_box)
    
    output = Output()
    display(output)
    


In [7]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Create Client
# -----------------------------------------------------------------------------------------------------------------------
def CreateClient():
    def post_client(b):
        with output:
            client = Post_Client(fname.value, lname.value, age.value, memberyear.value)
            if client.status_code == 201:
                output.clear_output()
                print("Client created successfully")
                print("Client Data:")
                print(client.json())
            else:
                output.clear_output()
                print("Invalid client data")
    
    operation = HTML(f"<p style='font-size:24px;'>Create Client</p>")
    fname = Text(placeholder="---", description="First Name")
    lname = Text(placeholder="---", description="Last Name")
    age = BoundedIntText(min=0, step=1, description="Age")
    memberyear = BoundedIntText(min=0, step=1, description="Member Year")
    create_button = Button(description="Create Client")
    create_button.on_click(post_client)
    create_box = VBox([operation, fname, lname, age, memberyear, create_button])
    display(create_box)
    output  = Output()
    display(output)


In [8]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Delete Client
# -----------------------------------------------------------------------------------------------------------------------
def DeleteClient():
    def delete_client(b):
        with output:
            client = Delete_Client(cid.value)
            if client.status_code == 200:
                output.clear_output()
                print("Client deleted successfully")
            elif client.status_code == 404:
                output.clear_output()
                print("Client not found")
                
            elif client.status_code == 400:
                output.clear_output()
                print("Client ID cannot be empty or Client has reservations")
            else:
                output.clear_output()
                print("Internal error")
            
    operation = HTML(f"<p style='font-size:24px;'>Delete Client</p>")
    
    cid = BoundedIntText(min=-1,max = 9999999999999,step=1, description="Client Id")
    delete_button = Button(description="Delete Client")
    delete_button.on_click(delete_client)
    create_box = VBox([operation, cid, delete_button])
    display(create_box)
    
    output = Output()
    display(output)

In [9]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Update Client
# -----------------------------------------------------------------------------------------------------------------------
def UpdateClient():
    def put_client(b):
        with output:
            client = Put_Client(cid.value, fname.value, lname.value, age.value, memberyear.value)
            if client.status_code == 201:
                output.clear_output()
                print("Client updated successfully")
                print("Client Data:")
                print(client.json())
            elif client.status_code == 404:
                output.clear_output()
                print("Client not found")
            else:
                output.clear_output()
                print("Invalid Client Data")
            
    operation = HTML(f"<p style='font-size:24px;'>Update Client</p>")
    fname = Text(placeholder="---", description="First Name")
    cid = BoundedIntText(min=-1,max = 9999999999999,step=1, description="Client Id")
    lname = Text(placeholder="---", description="Last Name")
    age = BoundedIntText(min=0,max = 200, step=1, description="Age")
    memberyear = BoundedIntText(min=0, step=1, description="Member Year")
    update_button = Button(description="Update Client")
    update_button.on_click(put_client)
    
    create_box = VBox([operation, cid, fname, lname, age, memberyear, update_button])
    display(create_box)
    
    output = Output()
    display(output)

In [10]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Create Employee with Account
# -----------------------------------------------------------------------------------------------------------------------
def CreateEmployee():
    def post_employee(b):
        with output:
            employee = Post_Employee(hid.value, position.value, salary.value, fname.value, lname.value, age.value, username.value, password.value)
            if employee.status_code == 201:
                output.clear_output()
                print("Employee created successfully")
                print("Employee Data:")
                print(employee.json())
            elif employee.json()['Error'] == "Invalid Data":
                output.clear_output()
                print("Invalid Data")
            elif employee.json()['Error'] == "Incorrect position for the hotel id.":
                output.clear_output()
                print("Incorrect position for the hotel id")
            elif employee.json()['Error'] == "Hotel not found":
                output.clear_output()
                print("Hotel not found")
            elif employee.json()['Error'] == "Invalid salary range for Regular position":
                output.clear_output()
                print("Invalid salary range for Regular position")
            elif employee.json()['Error'] == "Invalid salary range for Supervisor position":
                output.clear_output()
                print("Invalid salary range for Supervisor position")
            elif employee.json()['Error'] == "Invalid salary range for Administrator position":
                output.clear_output()
                print("Invalid salary range for Administrator position")
            elif employee.json()['Error'] == ("The employee was not created because the "
                                              "Login username exist."):
                output.clear_output()
                print("The employee was not created because the Login username exist.")
            else:
                output.clear_output()
                print("Create Failed")
    
    operation = HTML(f"<p style='font-size:24px;'>Create Employee</p>")
    hid = BoundedIntText(min=-1,step=1, description="Hotel ID")
    position = Dropdown(description="Position", options=['Regular','Supervisor','Administrator'])
    salary = BoundedIntText(min=0,step=0.01, max=1000000000, description="Salary")
    fname = Text(placeholder="---", description="First Name")
    lname = Text(placeholder="---", description="Last Name")
    age = BoundedIntText(min=0,step=1, max=1000000000, description="Age")
    username = Text(placeholder="---", description="Username")
    password = Text(placeholder="---", description="Password")
    create_button = Button(description="Create Employee")
    create_button.on_click(post_employee)
    
    create_box = VBox([operation, hid, position, salary, fname, lname, age, username, password, create_button])
    display(create_box)
    
    output = Output()
    display(output)

In [11]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Delete Employee with Account?
# -----------------------------------------------------------------------------------------------------------------------
####################### Delete Employee ##########################
def DeleteEmployee():
    def delete_employee(b):
        with output:
            employee = Delete_Employee(eid.value)
            if employee.status_code == 200:
                output.clear_output()
                print("Employee and account deleted successfully")
            elif employee.status_code == 404:
                output.clear_output()
                print("Employee not found")
            else:
                output.clear_output()
                print("Delete Failed")
            
    operation = HTML(f"<p style='font-size:24px;'>Delete Employee</p>")
    eid = BoundedIntText(min=-1,step=1, max=10000, description="Employee ID")
    delete_button = Button(description="Delete")
    delete_button.on_click(delete_employee)
    
    create_box = VBox([operation, eid, delete_button])
    display(create_box)
    
    output = Output()
    display(output)

In [12]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Update Employee
# -----------------------------------------------------------------------------------------------------------------------
####################### Update Employee ##########################


def UpdateEmployee():
    def put_employee(b):
        with output:
            employee = Put_Employee(eid.value,hid.value, position.value, salary.value, fname.value, lname.value, age.value)
            if employee.status_code == 200:
                output.clear_output()
                print("Employee updated successfully")
            elif employee.json()['Error'] == "Invalid Data":
                output.clear_output()
                print("Invalid Data")
            elif employee.json()['Error'] == "Hotel not found":
                output.clear_output()
                print("Hotel not found")
            elif employee.json()['Error'] == "Invalid salary range for Regular position":
                output.clear_output()
                print("Invalid salary range for Regular position")
            elif employee.json()['Error'] == "Invalid salary range for Supervisor position":
                output.clear_output()
                print("Invalid salary range for Supervisor position")
            elif employee.json()['Error'] == "Invalid salary range for Administrator position":
                output.clear_output()
                print("Invalid salary range for Administrator position")
            elif employee.json()['Error'] == "Employee not found":
                output.clear_output()
                print("Employee not found")
            else:
                output.clear_output()
                print("Update Failed")
            
    operation = HTML(f"<p style='font-size:24px;'>Update Employee</p>")
    
    eid = BoundedIntText(min=-1, max=10000000000000, step=1, description="Employee ID")
    hid = BoundedIntText(min=-1,step=1, description="Hotel ID")
    position = Dropdown(description="Position", options=['Regular','Supervisor','Administrator'])
    salary = BoundedIntText(min=0,step=0.01, max=1000000000, description="Salary")
    fname = Text(placeholder="---", description="First Name")
    lname = Text(placeholder="---", description="Last Name")
    age = BoundedIntText(min=0,step=1, max=1000000000, description="Age")
    update_button = Button(description="Update Employee")
    update_button.on_click(put_employee)
    
    create_box = VBox([operation, eid, hid, position, salary, fname, lname, age, update_button])
    display(create_box)
    
    output = Output()
    display(output)


In [13]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Create Hotel
# -----------------------------------------------------------------------------------------------------------------------
def CreateHotel():
    def post_hotel(b):
        with output:
            hotel = Post_Hotel(chid.value, hname.value, hcity.value)
            if hotel.status_code == 201:
                output.clear_output()
                print("Hotel created successfully")
                print("Hotel Data:")
                print(hotel.json())
            elif hotel.status_code == 404:
                output.clear_output()
                print("Chain does not exist")
            else:
                output.clear_output()
                print("Invalid hotel data")
    
    operation = HTML(f"<p style='font-size:24px;'>Create Hotel</p>")
    
    chid = BoundedIntText(min=-1,step=1, description="Chain Id")
    hname = Text(placeholder="---", description="Hotel Name")
    hcity = Text(placeholder="---", description="Hotel City")
    create_button = Button(description="Create Hotel")
    create_button.on_click(post_hotel)
    
    create_box = VBox([operation, chid, hname, hcity, create_button])
    display(create_box)
    
    output = Output()
    display(output)

In [14]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Delete Hotel
# -----------------------------------------------------------------------------------------------------------------------
def DeleteHotel():
    def delete_hotel(b):
        with output:
            hotel = Delete_Hotel(hid.value)
            if hotel.status_code == 200:
                output.clear_output()
                print("Hotel deleted successfully")
            elif hotel.status_code == 404:
                output.clear_output()
                print("Hotel not found")
            elif hotel.status_code == 400:
                output.clear_output()
                print("Hotel is referenced")
            else:
                output.clear_output()
                print("Delete Failed")
            
    operation = HTML(f"<p style='font-size:24px;'>Delete Hotel</p>")
    
    hid = BoundedIntText(min=-1,step=1, description="Hotel Id")
    delete_button = Button(description="Delete")
    delete_button.on_click(delete_hotel)
    
    create_box = VBox([operation, hid, delete_button])
    display(create_box)
    
    output = Output()
    display(output)

In [15]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Update Hotel
# -----------------------------------------------------------------------------------------------------------------------
def UpdateHotel():
    def put_hotel(b):
        with output:
            hotel = Put_Hotel(hid.value, chid.value, hname.value, hcity.value)
            if hotel.status_code == 202:
                output.clear_output()
                print("Hotel updated successfully")
                print("Hotel Data:")
                print(hotel.json())
            elif hotel.status_code == 404:
                output.clear_output()
                print("Hotel or Chain not found")
            else:
                output.clear_output()
                print("Invalid Hotel Data")
            
    operation = HTML(f"<p style='font-size:24px;'>Update Hotel</p>")
    
    hid = BoundedIntText(min=-1,step=1, description="Hotel Id")
    chid = BoundedIntText(min=-1,step=1, description="Chain Id")
    hname = Text(placeholder="---", description="Hotel Name")
    hcity = Text(placeholder="---", description="Hotel City")
    update_button = Button(description="Update Hotel")
    update_button.on_click(put_hotel)
    
    create_box = VBox([operation, hid, chid, hname, hcity, update_button])
    display(create_box)
    
    output = Output()
    display(output)

In [16]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Create Login
# -----------------------------------------------------------------------------------------------------------------------
def CreateLogin():
    def post_login(b):
        with output:
            login = Post_LogIn(eid.value, username.value, password.value)
            if login.status_code == 201:
                output.clear_output()
                print("Login created successfully")
                print("Login Data:")
                print(login.json())
            elif login.json()['Error'] == "Invalid Data":
                output.clear_output()
                print("Invalid Data")
            elif login.json()['Error'] == "Employee doesn't work with us.":
                output.clear_output()
                print("Employee doesn't work with us.")
            elif login.json()['Error'] == "Login could not be post because the username exist.":
                output.clear_output()
                print("Login could not be post because the username exist.")
            elif login.json()['Error'] == "Unexpected attribute values.":
                output.clear_output()
                print("Unexpected attribute values")
            elif login.json()['Error'] == "Employee have login account":
                output.clear_output()
                print("Employee have login account")
            else:
                output.clear_output()
                print("Create Failed")
    
    operation = HTML(f"<p style='font-size:24px;'>Create Login</p>")
    eid = BoundedIntText(min=-1,max=10000000000000, step=1, description="Employee ID")
    username = Text(placeholder="---", description="Username")
    password = Text(placeholder="---", description="Password")
    create_button = Button(description="Create Login")
    create_button.on_click(post_login)
    
    create_box = VBox([operation, eid, username, password, create_button])
    display(create_box)
    
    output = Output()
    display(output)

In [17]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Delete Login
# -----------------------------------------------------------------------------------------------------------------------
def DeleteLogin():
    def delete_login(b):
        with output:
            login = Delete_Login(lid.value)  
            if login.status_code == 200:
                output.clear_output()
                print(f"Login with lid {lid.value} deleted successfully")
            elif login.json()['Error'] == "This account doesn't exist.":
                output.clear_output()
                print(f"Login with lid {lid.value} not found")
            else:
                output.clear_output()
                print("Delete Failed")
            
    operation = HTML(f"<p style='font-size:24px;'>Delete Login</p>")  
    lid = BoundedIntText(min=-1,max=1000000000, step=1, description="Login ID") 
    delete_button = Button(description="Delete")
    delete_button.on_click(delete_login)
    
    create_box = VBox([operation, lid, delete_button])  
    display(create_box)
    
    output = Output()
    display(output)

In [18]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Update Login
# -----------------------------------------------------------------------------------------------------------------------
####################### Update Login ##########################

def UpdateLogin():
    def put_login(b):
        with output:
            login = Put_Login(lid.value, username.value, password.value)
            if login.status_code == 200:  
                output.clear_output()
                print(f"Login with lid {lid.value} updated successfully")
            elif login.json()['Error'] == "Invalid Data entered":
                output.clear_output()
                print("Invalid Data entered")
            elif login.json()['Error'] == "Login could not be put because the username exist.":
                output.clear_output()
                print("Login could not be put because the username exist.")
            elif login.json()['Error'] == "This account doesn't exist. Please create it first!":
                output.clear_output()
                print("This account doesn't exist. Please create it first.")
            elif login.json()['Error'] == "Username or Password is incorrect. Try again.":
                output.clear_output()
                print("Username or Password is incorrect. Try again.")
            else:
                output.clear_output()
                print("Update Failed")
    
    operation = HTML(f"<p style='font-size:24px;'>Update Login</p>")
    lid = BoundedIntText(min=-1, max=10000000000000, step=1, description="Login ID")
    username = Text(placeholder="---", description="Username")
    password = Text(placeholder="---", description="Password")
    update_button = Button(description="Update Login")
    update_button.on_click(put_login)
    
    create_box = VBox([operation, lid, username, password, update_button])
    display(create_box)
    
    output = Output()
    display(output)

In [19]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Create Reserve with Roomunavailable
# -----------------------------------------------------------------------------------------------------------------------
def CreateReserve():
    def post_reserve(b):
        output.clear_output()
        with output:
            reserve = Post_Reserve(eid.value, rid.value, guests.value, startdate.value, enddate.value, clid.value, payment.value)
            if reserve.status_code == 201:
                output.clear_output()
                print("Reserve created successfully")
                print("Reserve Data:")
                print(reserve.json())
            elif reserve.status_code == 404:
                output.clear_output()
                print("Employee or Room not found")
                
            else :
                output.clear_output()
                print(f"Invalid reserve data")

    operation = HTML(f"<p style='font-size:24px;'>Create Reserve</p>")
    eid = BoundedIntText(min=-1,max = 5000000,step=1, description="Employee Id")
    rid = BoundedIntText(min=-1,max = 5000000,step=1, description="Room Id")
    guests = BoundedIntText(min=-1,step=1, description="Guests")
    startdate = Text(placeholder="---", description="Start Date")
    enddate = Text(placeholder="---", description="End Date")
    clid = BoundedIntText(min=-1,step=1, description="Client Id")
    payment = Dropdown(options=["credit card","debit card","pear day"], description="Payment")
    create_button = Button(description="Create Reserve")
    create_button.on_click(post_reserve)
    
    create_box = VBox([operation, eid, rid, guests, startdate, enddate, clid, payment, create_button])
    display(create_box)
    
    output = Output()
    display(output)

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Delete Reserve with Roomunavailable
# -----------------------------------------------------------------------------------------------------------------------
def DeleteReserve():
    def delete_reserve(b):
        with output:
            reserve = Delete_Reserve(rid.value)
        if reserve.status_code == 200:
            output.clear_output()
            print("Reserve deleted successfully")
        elif reserve.status_code == 404:
            output.clear_output()
            print("Reserve not found")
        
        else:
            output.clear_output()
            print("Internal error, Error Deleting")
    
    operation = HTML(f"<p style='font-size:24px;'>Delete Reserve</p>")
    
    rid = BoundedIntText(min=-1,max = 999999999999,step=1, description="Reserve Id")
    delete_button = Button(description="Delete Reserve")
    delete_button.on_click(delete_reserve)
    
    create_box = VBox([operation, rid, delete_button])
    display(create_box)
    
    output = Output()
    display(output)

In [21]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Update Reserve
# -----------------------------------------------------------------------------------------------------------------------
def UpdateReserve():
    def put_reserve(b):
        with output:
            reserve = Put_Reserve(eid.value,reid.value,rid.value,guests.value,startdate.value,enddate.value,clid.value,payment.value)
            if reserve.status_code == 201:
                output.clear_output()
                print("Reserve updated successfully")
                print("Reserve Data:")
                print(reserve.json())
            elif reserve.status_code == 404:
                output.clear_output()
                print("Reserve or Employee not found")
            elif reserve.status_code == 400:
                output.clear_output()
                print("Invalid Reserve Data")
            else:
                output.clear_output()
                print("Internal error, Error Updating")
            
    operation = HTML(f"<p style='font-size:24px;'>Update Reserve</p>")
    reid = BoundedIntText(min=-1,max=5000000,step=1, description="Reserve Id")
    eid = BoundedIntText(min=-1,max = 5000000,step=1, description="Employee Id")
    rid = BoundedIntText(min=-1,max = 5000000,step=1, description="Room Id")
    guests = BoundedIntText(min=-1,step=1, description="Guests")
    startdate = Text(description="Start Date")
    enddate = Text(description="End Date")
    clid = BoundedIntText(min=-1,step=1, description="Client Id")
    payment = Dropdown(options=["cash","credit card", "debit card", "pear day",], description="Payment")

    update_button = Button(description="Update Reserve")
    update_button.on_click(put_reserve)
    
    create_box = VBox([operation, eid,reid, rid, guests, startdate, enddate, clid, payment, update_button])
    display(create_box)
    
    output = Output()
    display(output)

In [22]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Create Room
# -----------------------------------------------------------------------------------------------------------------------
def CreateRoom():
    def post_room(b):
        with output:
            room = Post_Room(hid.value, rdid.value, rprice.value)
            if room.status_code == 201:
                output.clear_output()
                print("Room created successfully")
                print("Room Data:")
                print(room.json())
            elif room.status_code == 404:
                output.clear_output()
                print("Hotel or Room Description Not Found")
            else:
                output.clear_output()
                print("Invalid data")
                
    operation = HTML(f"<p style='font-size:24px;'>Create Room</p>")
    hid = BoundedIntText(min=-1,step=1, description="Hotel ID")
    rdid = BoundedIntText(min=0,step=1, description="Room Description ID")
    rprice = BoundedIntText(min=0,step=0.01, description="Room Price")
    create_button = Button(description="Create Room")
    create_button.on_click(post_room)
    
    create_box = VBox([operation, hid, rdid, rprice, create_button])
    display(create_box)
    
    output = Output()
    display(output)

In [23]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Delete Room
# -----------------------------------------------------------------------------------------------------------------------
def DeleteRoom():
    def delete_room(b):
        with output:
            room = Delete_Room(rid.value)
            if room.status_code == 200:
                output.clear_output()
                print("Room deleted successfully")
            elif room.status_code == 404:
                output.clear_output()
                print("Room not found")
            elif room.status_code == 400:
                output.clear_output()
                print("Room is referenced")
            else:
                output.clear_output()
                print("Delete Failed")
    
    operation = HTML(f"<p style='font-size:24px;'>Delete Room</p>")
    rid = BoundedIntText(min=0,step=1, max=10000, description="Room ID")
    delete_button = Button(description="Delete Room")
    delete_button.on_click(delete_room)
    
    create_box = VBox([operation, rid, delete_button])
    display(create_box)
    
    output = Output()
    display(output)

In [24]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Update Room
# -----------------------------------------------------------------------------------------------------------------------
def UpdateRoom():
    def put_room(b):
        with output:
            room = Put_Room(rid.value, hid.value, rdid.value, rprice.value)
            if room.status_code == 202:
                output.clear_output()
                print("Room updated successfully")
                print("Room Data:")
                print(room.json())
            elif room.status_code == 404:
                output.clear_output()
                print("Room or Hotel or Room Description not found")
            else:
                output.clear_output()
                print("Invalid Data")
    
    operation = HTML(f"<p style='font-size:24px;'>Update Room</p>")
    rid = BoundedIntText(min=0,step=1, max=10000, description="Room Id")
    hid = BoundedIntText(min=0,step=1, max=10000, description="Hotel Id")
    rdid = BoundedIntText(min=0,step=1, max=10000, description="Room Description ID")
    rprice = BoundedIntText(min=0,step=1, max=10000, description="Room Price")
    update_button = Button(description="Update Room")
    update_button.on_click(put_room)
    
    create_box = VBox([operation, rid, hid, rdid, rprice, update_button])
    display(create_box)
    
    output = Output()
    display(output)

In [25]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Create Roomdescription
# -----------------------------------------------------------------------------------------------------------------------
def CreateRoomdescription():
    def post_roomdescription(b):
        with output:
            roomdescription = Post_RoomDescription(capacity.value, ishandicap.value, rname.value, rtype.value)
            if roomdescription.status_code == 201:
                output.clear_output()
                print("Room Description Created")
                print(roomdescription.json())
            elif roomdescription.json()['Error'] == 'Invalid Room Name.':
                output.clear_output()
                print("Invalid Room Name.")
            elif roomdescription.json()['Error'] == 'Invalid Capacity':
                output.clear_output()
                print('Invalid Capacity')
            elif roomdescription.json()['Error'] == 'Invalid Room Type':
                output.clear_output()
                print('Invalid Room Type')
            else:
                output.clear_output()
                print('Internal Server Error')
        
    operation = HTML(f"<p style='font-size:24px;'>Create Room Description</p>")
    capacity = Dropdown(description="Capacity", options=[1,2,4,6,8], value=1)
    ishandicap = Checkbox(value=False, description="Is handicap?")
    rname = Dropdown(description='Room Name', options=['Standard', 'Standard Queen', 'Standard King', 'Double Queen', 'Double King', 'Triple King', 'Executive Family', 'Presidential'])
    rtype = Dropdown(description='Room Type', options=['Basic', 'Premium', 'Deluxe', 'Suite'])
    create_button = Button(description="Create Room Description")
    create_button.on_click(post_roomdescription)
        
    create_box = VBox([operation, capacity, ishandicap, rname, rtype, create_button])
    
    display(create_box)
        
    output = Output()
    display(output)

In [26]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Delete Roomdescription
# -----------------------------------------------------------------------------------------------------------------------
def DeleteRoomdescription():
    def delete_roomdescription(b):
        with output:
            roomdescription = Delete_RoomDescription(rdid.value)
            if roomdescription.status_code == 200:
                output.clear_output()
                print("Room deleted successfully")
            elif roomdescription.status_code == 404:
                output.clear_output()
                print("Room not found")
            elif roomdescription.status_code == 400:
                output.clear_output()
                print("Room is referenced")
            else:
                output.clear_output()
                print("Delete Failed")
        
    operation = HTML(f"<p style='font-size:24px;'>Delete Room Description</p>")
    rdid = BoundedIntText(min=0,step=1, max=10000, description="Room Description ID")
    delete_button = Button(description="Delete Room Description")
    delete_button.on_click(delete_roomdescription)
    
    create_box = VBox([operation, rdid, delete_button])
    display(create_box)
    
    output = Output()
    display(output)

In [27]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Update Roomdescription
# -----------------------------------------------------------------------------------------------------------------------
def UpdateRoomdescription():
    def put_roomdescription(b):
        with output:
            roomdescription = Put_RoomDescription(rdid.value, capacity.value, ishandicap.value, rname.value, rtype.value)
            if roomdescription.status_code == 200:
                output.clear_output()
                print("Room Description updated successfully")
                print("Room Description Data:")
                print(roomdescription.json())
            elif roomdescription.json()['Error'] == 'Room Description not found':
                output.clear_output()
                print("Room Description not found")
            elif roomdescription.json()['Error'] == 'Invalid Room Name.':
                output.clear_output()
                print("Invalid Room Name.")
            elif roomdescription.json()['Error'] == 'Invalid Capacity':
                output.clear_output()
                print('Invalid Capacity')
            elif roomdescription.json()['Error'] == 'Invalid Room Type':
                output.clear_output()
                print('Invalid Room Type')
            else:
                output.clear_output()
                print('Internal Server Error')
        
    operation = HTML(f"<p style='font-size:24px;'>Update Room Description</p>")
    rdid = BoundedIntText(min=0,step=0.01, description="Room Description ID")
    capacity = Dropdown(description="Capacity", options=[1,2,4,6,8], value=1)
    ishandicap = Checkbox(value=False, description="Is handicap?")
    rname = Dropdown(description='Room Name', options=['Standard', 'Standard Queen', 'Standard King', 'Double Queen', 'Double King', 'Triple King', 'Executive Family', 'Presidential'])
    rtype = Dropdown(description='Room Type', options=['Basic', 'Premium', 'Deluxe', 'Suite'])
    update_button = Button(description="Update Room")
    update_button.on_click(put_roomdescription)
        
    create_box = VBox([operation, rdid, capacity, ishandicap, rname, rtype, update_button])
    
    display(create_box)
        
    output = Output()
    display(output)

In [28]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Create Roomunavailable
# -----------------------------------------------------------------------------------------------------------------------
def CreateRoomunavailable(id):
    def post_roomunavailable(b):
        with output:
            roomunavailable = Post_RoomUnavailable(id, rid.value, startdate.value, enddate.value)
            if roomunavailable.status_code == 201:
                output.clear_output()
                print('Room Unavailable created successfully')
                print(roomunavailable.json())
            elif roomunavailable.json()["Error"] == "The employee position is not Supervisor and Administrator":
                output.clear_output()
                print('The employee position is not Supervisor and Administrator')
            elif roomunavailable.json()["Error"] == "Room not found in the hotel":
                output.clear_output()
                print('Room not found in the hotel')
            elif roomunavailable.json()["Error"] == "End date must be greater than start date":
                output.clear_output()
                print('End date must be greater than start date')
            elif roomunavailable.json()["Error"] == "Room is not available during the selected dates.":
                output.clear_output()
                print('Room is not available during the selected dates.')
            elif roomunavailable.json()["Error"] == "The room could not be separated.":
                output.clear_output()
                print('The room could not be separated.')
            else:
                output.clear_output()
                print("Internal Server Error")
        
    operation = HTML(f"<p style='font-size:24px;'>Create Room Unavailable</p>")
    rid = BoundedIntText(min=0,step=1, max=7777, description="Room ID")
    startdate = Text(placeholder="---", description='Start Date')
    enddate = Text(placeholder="---", description='End Date')
    create_button = Button(description="Create Room Description")
    create_button.on_click(post_roomunavailable)
        
    create_box = VBox([operation, rid, startdate, enddate, create_button])
    display(create_box)
        
    output = Output()
    display(output)

In [29]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Delete Roomunavailable
# -----------------------------------------------------------------------------------------------------------------------
def DeleteRoomunavailable(id):
    def delete_roomunavailable(b):
        with output:
            roomunavailable = Delete_RoomUnavailable(id, ruid.value)
            if roomunavailable.status_code == 200:
                output.clear_output()
                print("Room unavailable deleted successfully")
            elif roomunavailable.json()['Error']==  "The employee position is not Supervisor and Administrator":
                output.clear_output()
                print("The employee position is not Supervisor and Administrator")
            elif roomunavailable.json()['Error']== "RoomUnavailable not found.":
                output.clear_output()
                print("RoomUnavailable not found.")
            elif roomunavailable.json()['Error']== "Room Unavailable is referenced":
                output.clear_output()
                print("Room Unavailable is referenced")
            else:
                output.clear_output()
                print("Internal Server Error")
    
    operation = HTML(f"<p style='font-size:24px;'>Delete Room Unavailable</p>")
    ruid = BoundedIntText(min=0,step=1, max=7777, description="Room Unavailable ID")
    delete_button = Button(description="Delete Room Unavailable")
    delete_button.on_click(delete_roomunavailable)
    
    create_box = VBox([operation, ruid, delete_button])
    display(create_box)
    
    output = Output()
    display(output)

In [30]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Update Roomunavailable
# -----------------------------------------------------------------------------------------------------------------------
def UpdateRoomunavailable(id):
    def put_roomunavailable(b):
        with output:
            roomunavailable = Put_RoomUnavailable(id, ruid.value, rid.value, startdate.value, enddate.value)
            if roomunavailable.status_code == 200:
                output.clear_output()
                print("Room unavailable updated successfully")
                print(roomunavailable.json())
            elif roomunavailable.json()["Error"] == "The employee position is not Supervisor and Administrator":
                output.clear_output()
                print('The employee position is not Supervisor and Administrator')
            elif roomunavailable.json()["Error"] == "RoomUnavailable not found.":
                output.clear_output()
                print('RoomUnavailable not found.')
            elif roomunavailable.json()["Error"] == ("The unavailable room is referenced to a reservation. "
                                                     "To upgrade the unavailable room, use the upgrade reservation."):
                output.clear_output()
                print('The unavailable room is referenced to a reservation')
            elif roomunavailable.json()["Error"] == "Room not found in the hotel":
                output.clear_output()
                print('Room not found in the hotel')
            elif roomunavailable.json()["Error"] == "End date must be greater than start date":
                output.clear_output()
                print('End date must be greater than start date')
            elif roomunavailable.json()["Error"] == "Room is not available during the selected dates.":
                output.clear_output()
                print('Room is not available during the selected dates.')
            else:
                output.clear_output()
                print("Internal Server Error")

    operation = HTML(f"<p style='font-size:24px;'>Update Room Unavailable</p>")
    ruid = BoundedIntText(min=0,step=1, max=7777, description="Room Unavailable ID")
    rid = BoundedIntText(min=0,step=1, max=7777, description="Room ID")
    startdate = Text(placeholder="---", description='Start Date')
    enddate = Text(placeholder="---", description='End Date')
    update_button = Button(description="Update Room Unavailable")
    update_button.on_click(put_roomunavailable)
    
    create_box = VBox([operation, ruid, rid, startdate, enddate, update_button])
    
    display(create_box)
        
    output = Output()
    display(output)

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Global Statistics /most/revenue
# -----------------------------------------------------------------------------------------------------------------------
def MostRevenue(id):
    items = see_mostrevenue(id)
    if not items:
        print("Access not granted")
    else:
        datas = items.json()['Most_Revenues']
        
        # ** x axis values
        x = [str(x['chid']) for x in datas]
        # ** y axis values
        y = [float(y['total revenue']) for y in datas]
            
        # ** creating the bar plot
        plt.bar(x, y, color='black', bottom=0)
            
        plt.ylim(0, max(y) + min(y))
            
        # ** naming the x-axis
        plt.xlabel('x - axis = Chains')
        # ** naming the y-axis
        plt.ylabel('y - axis = Total Revenues in millions')
            
        # ** giving a title to my graph
        plt.title('Top 3 chains with the highest total revenue')
            
        # ** Tight layout to avoid overlap
        plt.tight_layout()
            
        # ** function to show the plot
        plt.show()
        
        # ** See data
        json_str = json.dumps(datas, indent=2)
        lines = json_str.split('\n')
        for line in lines:
            print(line)

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Global Statistics /paymentmethod
# -----------------------------------------------------------------------------------------------------------------------
def PaymentMethod(id):
    items = see_paymentmethod(id)
    if not items:
        print("Access not granted")
    else:
        datas = items.json()['Payment_Method_Percentage']
            
        # ** x axis values
        x = [str(method['paymentmethod']) for method in datas]
        # ** y axis values
        y = [float(percentage['reservation percentage']) for percentage in datas]
            
        # ** creating the bar plot
        plt.bar(x, y, color='black')
            
        plt.ylim(0, max(y)+5)
            
        # ** naming the x-axis
        plt.xlabel('x - axis = Payment Method')
        # ** naming the y-axis
        plt.ylabel('y - axis = Reservation Percentage')
            
        # ** giving a title to my graph
        plt.title('Total reservation percentage by payment method')
            
        # ** Tight layout to avoid overlap
        plt.tight_layout()
            
        # ** function to show the plot
        plt.show()
        
        # ** See data
        json_str = json.dumps(datas, indent=2)
        lines = json_str.split('\n')
        for line in lines:
            print(line)

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Global Statistics /least/rooms
# -----------------------------------------------------------------------------------------------------------------------
####################### /least/rooms ##########################

def LeastRoomsChains(id):
    def graph():
        items = see_least_rooms_chains(id)
        if not items:
            output.clear_output()
            print("Access not granted")
        else:
            datas = items.json()['Top_Three_Chains']
            
            x = [chain['cname'] for chain in datas]
            y = [int(chain['room_count']) for chain in datas]
            
            plt.bar(x, y, color='black')
            
            plt.ylim(0, max(y)+5)
            
            plt.xlabel('x - axis = Chain Name')
            plt.ylabel('y - axis = Room Count')
            plt.title('Top 3 Chains with Least Rooms')
            plt.xticks(rotation=45, ha='right')  
            plt.show()
            
            # ** See data
            json_str = json.dumps(datas, indent=3)
            lines = json_str.split('\n')
            for line in lines:
                print(line)

    operation = HTML(f"<p style='font-size:24px;'>Top 3 Chains with Least Rooms</p>")
    graph()

    output = Output()
    display(output)

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Global Statistics /most/capacity
# -----------------------------------------------------------------------------------------------------------------------
####################### /most/capacity ##########################

def MostCapacityPerChain(id):
    def graph():
        items = see_most_capacity_per_chain(id)
        if not items:
            output.clear_output()
            print("Access not granted")
        else:
            datas = items.json()['Top_Five_Hotels_With_Most_Capacity']
            
            x = [hotel['hotel_name'] for hotel in datas]
            y = [int(hotel['total_capacity']) for hotel in datas]
            
            plt.bar(x, y, color='black')
            
            plt.ylim(0, max(y)+5)
            
            plt.xlabel('x - axis = Hotel Name')
            plt.ylabel('y - axis = Total Capacity')
            plt.title('Top 5 Hotels with Most Capacity')
            plt.xticks(rotation=45, ha='right')  
            plt.show()
            
            # ** See data
            json_str = json.dumps(datas, indent=3)
            lines = json_str.split('\n')
            for line in lines:
                print(line)
       

    operation = HTML(f"<p style='font-size:24px;'>Top 5 Hotels with Most Capacity</p>")
    graph()

    output = Output()
    display(output)


In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Global Statistics /most/reservation
# -----------------------------------------------------------------------------------------------------------------------
####################### /most/reservation ##########################

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Global Statistics /most/profitmonth
# -----------------------------------------------------------------------------------------------------------------------
####################### /most/profitmonth ##########################

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Local Statistics /hotel/<id>/handicaproom
# -----------------------------------------------------------------------------------------------------------------------
####################### handicaproom ##########################
# def Handicaproom():
#    def display_graph(b):
#       if user.position == "Regular":
#          items = handicap_room(user.hotel_id,user.eid)
         
#       else:
#          items = handicap_room(hid_num,user.eid)
#          print('aaaaa')
#       if not items:
#          print("No data found")
#       else:
#          data = items.json()['Top5_HandicapRoom']['rooms']

#          # Extract data for plotting
#          room_ids = [entry['room_id'] for entry in data]
#          total_reserves = [entry['total_reserves'] for entry in data]

#          # Create the bar chart
#          plt.figure(figsize=(10, 6))  # Adjust size as needed
#          plt.bar(room_ids, total_reserves, color='skyblue')

#          # Add labels and title
#          plt.xlabel('Room ID')
#          plt.ylabel('Total Reservations')
#          plt.title('Top 5 Handicap Rooms by Reservations')

#          # Show the plot
#          plt.xticks(room_ids, rotation=45)  # Rotate labels if they overlap
#          plt.show()
         
#    operation = HTML(f"<p style='font-size:24px;'>Top 5 Handicap Rooms</p>")
#    hid_num = BoundedIntText(min=1, max=9999999999999999999, value=1, description='Hotel ID:')
#    submit_btn = Button(description='Submit')
#    submit_btn.on_click(display_graph)
#    create_box = VBox([operation,hid_num, submit_btn])
#    display(create_box)
   
   
#    output = Output()
#    display(output)

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Local Statistics /hotel/<id>/leastreserve
# -----------------------------------------------------------------------------------------------------------------------
####################### leastreserve ##########################

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Local Statistics /hotel/<id>/mostcreditcard
# -----------------------------------------------------------------------------------------------------------------------
def MostCreditCard(id):
    def graph():
        items = see_mostcreditcard(hid.value, id)
        if not items:
            output.clear_output()
            print("Access not granted")
        else:
            datas = items.json()['Most_Credit_Cards']
            
            # ** Graph
            x = [str(clids['clid']) for clids in datas]
            y = [int(reserves['total reserves']) for reserves in datas]
            
            plt.bar(x, y, color='black')
            
            plt.ylim(0, max(y)+3)
            
            plt.title('Most Credit Cards')
            plt.ylabel('y - axis = Total Reserves')
            plt.xlabel('x - axis = Clients')
            plt.show()
            
            # ** See data
            print("Graph Data")
            json_str = json.dumps(datas, indent=2)
            lines = json_str.split('\n')
            for line in lines:
                print(line)

    def see_button(b):
        with output:
            mostcreditcard = see_mostcreditcard(hid.value, id)
            if not mostcreditcard:
                output.clear_output()
                print("Access not granted")
            else:
                if mostcreditcard.status_code == 200:
                    output.clear_output()
                    graph()
                elif mostcreditcard.status_code == 404:
                    output.clear_output()
                    print("Hotel ID invalid")
                elif mostcreditcard.status_code == 400:
                    output.clear_output()
                    print("Invalid Data")
                else:
                    output.clear_output()
                    print("Error")
    
    operation = HTML(f"<p style='font-size:24px;'>Top 5 clients under 30 that made the most reservation with a credit card.</p>")
    hid = BoundedIntText(min=-1,step=1, description="Hotel ID")
    see_mostcreditcard_button = Button(description="See Most Credit Card")
    see_mostcreditcard_button.on_click(see_button)
        
    create_box = VBox([operation, hid, see_mostcreditcard_button])
    display(create_box)
        
    output = Output()
    display(output)

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Local Statistics /hotel/<id>/highestpaid
# -----------------------------------------------------------------------------------------------------------------------
####################### highestpaid ##########################


def HighestPaidEmployees(id):
    def graph():
        items = see_highest_paid(hid.value, id)
        if not items:
            output.clear_output()
            print("Access not granted")
        else:
            datas = items.json()['Highest_Paid_Regular_Employees']
            
            x = [f"{name['fname']} {name['lname']}" for name in datas]
            y = [int(salary['salary']) for salary in datas]
            
            plt.bar(x, y, color='black')
            
            plt.ylim(0, max(y)+5)
            
            plt.xlabel('x - axis = Employee Name')
            plt.ylabel('y - axis = Salary')
            plt.title('Highest Paid Regular Employees')
            plt.xticks(rotation=45, ha='right')  
            plt.show()
            
            # ** See data
            json_str = json.dumps(datas, indent=3)
            lines = json_str.split('\n')
            for line in lines:
                print(line)

    def see_button(b):
        with output:
            highest_paid = see_highest_paid(hid.value, id)
            if not highest_paid:
                output.clear_output()
                print("Access not granted")
            else:
                if highest_paid.status_code == 200:
                    output.clear_output()
                    graph()
                elif highest_paid.status_code == 404:
                    output.clear_output()
                    print("Hotel ID invalid")
                elif highest_paid.status_code == 400:
                    output.clear_output()
                    print("Invalid Data")
                else:
                    output.clear_output()
                    print("Error")

    operation = HTML(f"<p style='font-size:24px;'>Highest Paid Regular Employees</p>")
    hid = BoundedIntText(min=-1,step=1, description="Hotel ID")
    see_highest_paid_button = Button(description="See Highest Paid")
    see_highest_paid_button.on_click(see_button)
    
    create_box = VBox([operation, hid, see_highest_paid_button])
    display(create_box)
    
    output = Output()
    display(output)

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Local Statistics /hotel/<id>/mostdiscount
# -----------------------------------------------------------------------------------------------------------------------
####################### mostdiscount ##########################

def TopDiscountClients(id):
    def graph():
        items = see_top_discount_clients(hid.value, id)
        if not items:
            output.clear_output()
            print("Access has not been granted")
        else:
            datas = items.json()['Most_Discounts']
            
            x = [f"{name['fname']} {name['lname']}" for name in datas]
            y = [float(discount['discount']) for discount in datas]  # Convert discount to float
            
            plt.bar(x, y, color='black')
            
            plt.ylim(0, max(y)+5)
            
            plt.xlabel('x - axis = Client Name')
            plt.ylabel('y - axis = Discount Amount')
            plt.title('Top 5 Clients with Most Discounts')
            plt.xticks(rotation=45, ha='right')  
            plt.show()
            
            # ** See data
            json_str = json.dumps(datas, indent=3)
            lines = json_str.split('\n')
            for line in lines:
                print(line)

    def see_button(b):
        with output:
            top_discount_clients = see_top_discount_clients(hid.value, id)
            if not top_discount_clients:
                output.clear_output()
                print("Access not granted") 
            else:
                if top_discount_clients.status_code == 200:
                    output.clear_output()
                    graph()
                elif top_discount_clients.status_code == 404:
                    output.clear_output()
                    print("Hotel ID invalid")
                elif top_discount_clients.status_code == 400:
                    output.clear_output()
                    print("Invalid Data")
                else:
                    output.clear_output()
                    print("Error")

    operation = HTML(f"<p style='font-size:24px;'>Top 5 Clients with Most Discounts</p>")
    hid = BoundedIntText(min=-1,step=1, description="Hotel ID")
    see_top_discount_clients_button = Button(description="Top Discount Clients")
    see_top_discount_clients_button.on_click(see_button)
    
    create_box = VBox([operation, hid, see_top_discount_clients_button])
    display(create_box)
    
    output = Output()
    display(output)


In [45]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Local Statistics /hotel/<id>/roomtype
# -----------------------------------------------------------------------------------------------------------------------
def RoomType(id):
    def graph():
        items = see_roomtype(hid.value, id)
        if not items:
            output.clear_output()
            print("Access not granted")
        else:
            datas = items.json()['Total_Hotel_Reserves']
            
            x = [str(types['room type']) for types in datas]
            y = [int(reserves['total reserves']) for reserves in datas]
            
            plt.bar(x, y, color='black')
            
            plt.ylim(0, max(y)+5)
            
            plt.xlabel('x - axis = Room Types')
            plt.ylabel('y - axis = Total Reserves')
            plt.title('Total reservation by room type')
            plt.show()
            
            # ** See data
            json_str = json.dumps(datas, indent=2)
            lines = json_str.split('\n')
            for line in lines:
                print(line)

    def see_button(b):
        with output:
            roomtypes = see_roomtype(hid.value, id)
            if not roomtypes:
                output.clear_output()
                print("Access not granted")
            else:
                if roomtypes.status_code == 200:
                    output.clear_output()
                    graph()
                elif roomtypes.status_code == 404:
                    output.clear_output()
                    print("Hotel ID invalid")
                elif roomtypes.status_code == 400:
                    output.clear_output()
                    print("Invalid Data")
                else:
                    output.clear_output()
                    print("Error")

    operation = HTML(f"<p style='font-size:24px;'>Total reservation by room type</p>")
    hid = BoundedIntText(min=-1,step=1, description="Hotel ID")
    see_roomtype_button = Button(description="See Room Types")
    see_roomtype_button.on_click(see_button)
    
    create_box = VBox([operation, hid, see_roomtype_button])
    display(create_box)
    
    output = Output()
    display(output)

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Local Statistics /hotel/<id>/leastguests
# -----------------------------------------------------------------------------------------------------------------------
def LeastGuests(id):
    def graph():
        items = see_leastguests(hid.value, id)
        if not items:
            output.clear_output()
            print("Access not granted")
        else:
            datas = items.json()['Least_Guests']
        
            # ** Graph
            x = [str(rooms['rid']) for rooms in datas]
            y = [float(ratios['ratio']) for ratios in datas]
            
            plt.bar(x, y, color='black')
            
            plt.ylim(0, max(y)+1)
            
            plt.title('Least Guests')
            plt.ylabel('y - axis = Guest-to-Capacity Ratio')
            plt.xlabel('x - axis = Rooms')
            plt.show()
            
            # ** See data
            print("Graph Data")
            json_str = json.dumps(datas, indent=2)
            lines = json_str.split('\n')
            for line in lines:
                print(line)
    
    def see_button(b):
        with output:
            leastguests = see_leastguests(hid.value, id)
            if not leastguests:
                output.clear_output()
                print("Access not granted")
            else:
                if leastguests.status_code == 200:
                    output.clear_output()
                    graph()
                elif leastguests.status_code == 404:
                    output.clear_output()
                    print("Hotel ID invalid")
                elif leastguests.status_code == 400:
                    output.clear_output()
                    print("Invalid Data")
                else:
                    output.clear_output()
                    print("Error")

    operation = HTML(f"<p style='font-size:24px;'>Top 3 rooms that were reserved that had the least guest-to-capacity ratio.</p>")
    hid = BoundedIntText(min=-1,step=1, description="Hotel ID")
    see_leastguests_button = Button(description="See Least Guests")
    see_leastguests_button.on_click(see_button)
    
    create_box = VBox([operation, hid, see_leastguests_button])
    display(create_box)
    
    output = Output()
    display(output)

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Region Display
# -----------------------------------------------------------------------------------------------------------------------
def Display_controller():
    match user.position:
        
        case "Regular":
            display(HTML(f"<p style='font-size:50px;'>CRUDS</p>"))
            display(HTML(f"<p style='font-size:30px;'>-----Reserve-----</p>"))
            display(CreateReserve())
            
            #Add local statistics
            display(HTML(f"<p style='font-size:50px;'>LOCAL STATISTICS</p>"))
            display(MostCreditCard(user.eid))
            display(HighestPaidEmployees(user.eid))
            display(TopDiscountClients(user.eid))
            display(RoomType(user.eid))
            display(LeastGuests(user.eid))
            
        case "Supervisor":
            display(HTML(f"<p style='font-size:50px;'>CRUDS</p>"))
            display(HTML(f"<p style='font-size:30px;'>-----RoomUnavailable-----</p>"))
            display(CreateRoomunavailable(user.eid))
           
            
            #Add local statistics
            display(HTML(f"<p style='font-size:50px;'>LOCAL STATISTICS</p>"))
            
            
            display(MostCreditCard(user.eid))
            display(HighestPaidEmployees(user.eid))
            display(TopDiscountClients(user.eid))
            display(RoomType(user.eid))
            display(LeastGuests(user.eid))
            
        case "Administrator":
            # Cruds
            display(HTML(f"<p style='font-size:50px;'>CRUDS</p>"))
            ## Chains
            display(HTML(f"<p style='font-size:30px;'>-----Chains-----</p>"))
            display(CreateChain())
            display(UpdateChain())
            display(DeleteChain())
            ## Hotel
            display(HTML(f"<p style='font-size:30px;'>-----Hotel-----</p>"))
            display(CreateHotel())
            display(UpdateHotel())
            display(DeleteHotel())
            ## Employee
            display(HTML(f"<p style='font-size:30px;'>-----Employee-----</p>"))
            display(CreateEmployee())
            display(UpdateEmployee())
            display(DeleteEmployee())
            ## Login
            display(HTML(f"<p style='font-size:30px;'>-----Login-----</p>"))
            display(CreateLogin())
            display(UpdateLogin())
            display(DeleteLogin())
            ## Room
            display(HTML(f"<p style='font-size:30px;'>-----Room-----</p>"))
            display(CreateRoom())
            display(UpdateRoom())
            display(DeleteRoom())
            ## RoomDescription
            display(HTML(f"<p style='font-size:30px;'>-----RoomDescription-----</p>"))
            display(CreateRoomdescription())
            display(UpdateRoomdescription())
            display(DeleteRoomdescription())
            ## RoomUnavailable
            display(HTML(f"<p style='font-size:30px;'>-----RoomUnavailable-----</p>"))
            display(CreateRoomunavailable(user.eid))
            display(UpdateRoomunavailable(user.eid))
            display(DeleteRoomunavailable(user.eid))
            ## Reserve
            display(HTML(f"<p style='font-size:30px;'>-----Reserve-----</p>"))
            display(CreateReserve())
            display(UpdateReserve())
            display(DeleteReserve())
            ## Client
            display(HTML(f"<p style='font-size:30px;'>-----Client-----</p>"))
            display(CreateClient())
            display(UpdateClient())
    
            #Add local statistics
            display(HTML(f"<p style='font-size:50px;'>LOCAL STATISTICS</p>"))
            
            
            display(MostCreditCard(user.eid))
            display(HighestPaidEmployees(user.eid))
            display(TopDiscountClients(user.eid))
            display(RoomType(user.eid))
            display(LeastGuests(user.eid))
            
            # Add Global statistics
            display(HTML(f"<p style='font-size:50px;'>GLOBAL STATISTICS</p>"))
            
            display(PaymentMethod(user.eid))
            display(MostRevenue(user.eid))
            display(LeastRoomsChains(user.eid))
            display(MostCapacityPerChain(user.eid))